In [89]:
import os
import urllib
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import calendar

In [90]:
path = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\TempComparison'
os.chdir(path)

csv = (path + r'\csv.csv')
f = open(csv,'w+')

In [91]:
urllib.request.urlretrieve(
    'http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=daily&sids=akr02&sdate=1992-07-01' +
    '&edate=2018-12-31&elems=tave,tmax,tmin',filename=csv)
coag_df = pd.read_csv(csv)
coag_df = coag_df.reset_index()
coag_df.columns = ['station','date','tave','tmax','tmin']
coag_df['date'] = pd.to_datetime(coag_df.date,errors='coerce')

coag_df.tave = 32 + (coag_df.tave * (9/5))
coag_df.tmax = 32 + (coag_df.tmax * (9/5))
coag_df.tmin = 32 + (coag_df.tmin * (9/5))

In [92]:
urllib.request.urlretrieve(
    'http://data.rcc-acis.org/StnData?sid=050109&sdate=1992-07-01&edate=2018-12-31' + 
    '&elems=avgt,maxt,mint&output=csv',filename=csv)
coop_df = pd.read_csv(csv)
coop_df = coop_df.reset_index()
coop_df.columns = ['date','avgt','maxt','mint']
coop_df['date'] = pd.to_datetime(coop_df.date,errors='coerce')
coop_df = coop_df.replace('M',np.nan,regex=True)

In [93]:
df = coop_df.merge(coag_df,left_index=True,right_index=True)
stuff = ['date_y','station']
df = df.drop(stuff,axis=1)
df.columns = ['date','avgt','maxt','mint','tave','tmax','tmin']


In [94]:
stuff = ['avgt','tave']
maxmin_df = df.drop(stuff,axis=1)
maxmin_df = maxmin_df.dropna()

In [95]:
maxmin_df['min_temp_diff'] = (maxmin_df.tmin.astype(float) - maxmin_df.mint.astype(float))
maxmin_df['max_temp_diff'] = (maxmin_df.tmax.astype(float) - maxmin_df.maxt.astype(float))
maxmin_df['station'] = 'akr02'
maxmin_df

,date,maxt,mint,tmax,tmin,min_temp_diff,max_temp_diff,station
31,1992-08-01,91,51,92.8580,52.7000,1.7000,1.8580,akr02
32,1992-08-02,90,58,85.6220,60.3860,2.3860,-4.3780,akr02
33,1992-08-03,84,60,86.0540,59.0900,-0.9100,2.0540,akr02
34,1992-08-04,83,55,92.0120,56.8400,1.8400,9.0120,akr02
35,1992-08-05,89,53,80.2580,54.8240,1.8240,-8.7420,akr02
36,1992-08-06,80,62,90.1040,60.4760,-1.5240,10.1040,akr02
37,1992-08-07,90,56,100.2920,61.1600,5.1600,10.2920,akr02
38,1992-08-08,95,60,77.5580,60.0080,0.0080,-17.4420,akr02
39,1992-08-09,94,59,81.2480,53.7980,-5.2020,-12.7520,akr02
40,1992-08-10,98,62,69.8720,54.8960,-7.1040,-28.1280,akr02


In [96]:
maxmin_df['month'] = maxmin_df.date.dt.month
maxmin_df['year'] = maxmin_df.date.dt.year

maxmin_df['maxt'] = maxmin_df.maxt.astype(float)
maxmin_df['mint'] = maxmin_df.mint.astype(float)

maxmin_df.to_csv(path + r'\akr02-coop_tempcomparison.csv')

In [9]:
monthly = maxmin_df.groupby(['month']).mean().reset_index()

monthly['month'] = monthly['month'].apply(lambda x: calendar.month_abbr[x])
monthly

,month,maxt,mint,tmax,tmin,min_temp_diff,max_temp_diff,year,station
0,Jan,41.435216,15.375415,44.829966,17.262022,1.886606,3.394750,2002.571429,akr02
1,Feb,43.625899,17.298561,54.166744,23.967257,6.668696,10.540845,2002.571942,akr02
2,Mar,54.157635,25.302135,60.204803,31.304287,6.002152,6.047167,2002.530378,akr02
3,Apr,61.482993,32.396259,68.700344,41.045903,8.649645,7.217351,2002.554422,akr02
4,May,71.245528,42.531707,82.041668,51.831677,9.299970,10.796140,2002.518699,akr02
5,Jun,82.546531,52.231810,88.092569,58.032447,5.800636,5.546037,2002.524535,akr02
6,Jul,90.224194,58.987097,87.930732,57.622065,-1.365032,-2.293461,2002.500000,akr02
7,Aug,87.524845,56.866460,79.543084,49.765558,-7.100901,-7.981761,2001.978261,akr02
8,Sep,78.683121,47.466561,67.302798,37.346837,-10.119724,-11.380323,2001.998408,akr02
9,Oct,65.126183,35.022082,54.634566,25.514705,-9.507377,-10.491617,2002.026814,akr02


In [59]:
tmin_fig = go.Figure(data=[go.Bar()],
                    layout=go.Layout(title=go.layout.Title(text='Min Temp Comparison'),
                                    xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Month')),
                                    yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Temperature (°F)'))))

tmin_fig.add_trace(go.Bar(x=monthly.month,y=monthly.tmin,name='AKR02'))
tmin_fig.add_trace(go.Bar(x=monthly.month,y=monthly.mint,name='Akron A 4 E'))

tmin_fig.show()